In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [2]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [3]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [4]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW


C:\Users\user\AppData\Local\Temp\ipykernel_20616\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_20616\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdj

In [5]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [6]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [7]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data
873,00713,統一 ( 10.09%)
874,00713,台灣大 ( 6.79%)
875,00713,遠傳 ( 5.87%)
876,00713,可成 ( 5.49%)
877,00713,華南金 ( 4.41%)
878,00713,亞泥 ( 4.1%)
879,00713,統一超 ( 3.97%)
880,00713,長榮航 ( 2.29%)
881,00713,崇越 ( 2.23%)
882,00713,東和鋼鐵 ( 2.2%)


In [8]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,0050,光電業,大立光 ( 0.53%)
1,0050,其他,中租-KY ( 0.44%)
2,0050,其他電子業,鴻海 ( 5.68%)
3,0050,半導體業,"台積電 ( 56.63%),聯發科 ( 4.36%),聯電 ( 1.29%),日月光投控 (..."
4,0050,塑膠工業,"南亞 ( 0.49%),台塑 ( 0.4%),台化 ( 0.3%)"
...,...,...,...
701,00961,金融保險,"玉山金 ( 5.04%),京城銀 ( 4.4%),上海商銀 ( 1.2%)"
702,00961,電子通路業,"大聯大 ( 3.98%),聯強 ( 1.15%)"
703,00961,電子零組件業,群光 ( 1.3%)
704,00961,電腦及週邊設備業,"新普 ( 1.85%),光寶科 ( 1.72%),和碩 ( 1.34%)"


In [9]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_20616\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers
200,00713,光電業,4.31,9.0
201,00713,其他,7.77,4.0
202,00713,其他電子業,7.68,5.0
203,00713,化學工業,0.96,19.0
204,00713,半導體業,7.07,6.0
205,00713,建材營造,1.34,15.0
206,00713,水泥工業,4.10,10.0
207,00713,汽車工業,0.92,20.0
208,00713,紡織纖維,1.26,16.0
209,00713,航運業,2.29,12.0


In [10]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,65.02,"台積電 ( 56.63%),聯發科 ( 4.36%),聯電 ( 1.29%),日月光投控 (...",金融保險,11.32,"富邦金 ( 1.68%),國泰金 ( 1.44%),中信金 ( 1.34%),兆豐金 ( 1...",其他電子業,5.68,鴻海 ( 5.68%)
1,0051,電子零組件業,13.25,"健策 ( 2.04%),台光電 ( 1.71%),嘉澤 ( 1.66%),群光 ( 1.29...",半導體業,12.76,"矽力*-KY ( 2.25%),京元電子 ( 1.88%),創意 ( 1.48%),力成 (...",電腦及週邊設備業,9.74,"仁寶 ( 1.96%),技嘉 ( 1.73%),微星 ( 1.66%),英業達 ( 1.58..."
2,0052,半導體業,77.88,"台積電 ( 65.62%),聯發科 ( 5.07%),聯電 ( 1.5%),日月光投控 ( ...",電腦及週邊設備業,8.32,"廣達 ( 2.14%),華碩 ( 1.07%),緯創 ( 0.8%),和碩 ( 0.54%)...",其他電子業,7.51,"鴻海 ( 6.69%),可成 ( 0.38%),鴻準 ( 0.24%),貿聯-KY ( 0.2%)"
3,0053,半導體業,64.34,"台積電 ( 52.02%),聯發科 ( 4.56%),日月光投控 ( 1.35%),聯電 (...",電腦及週邊設備業,8.98,"廣達 ( 2.26%),華碩 ( 0.81%),緯穎 ( 0.72%),緯創 ( 0.63%...",電子零組件業,8.04,"台達電 ( 2.73%),國巨 ( 0.67%),欣興 ( 0.46%),嘉澤 ( 0.42..."
4,0056,電腦及週邊設備業,25.73,"華碩 ( 3.56%),廣達 ( 2.95%),和碩 ( 2.86%),緯創 ( 2.66%...",半導體業,18.98,"聯發科 ( 3.25%),聯詠 ( 3.17%),聯電 ( 3.01%),瑞昱 ( 2.93...",金融保險,13.16,"兆豐金 ( 3.59%),中信金 ( 3.01%),永豐金 ( 2.91%),元大金 ( 2..."
5,0057,半導體業,60.55,"台積電 ( 51.46%),聯發科 ( 3.94%),聯電 ( 1.1%),日月光投控 ( ...",金融保險,10.56,"富邦金 ( 1.51%),國泰金 ( 1.33%),中信金 ( 1.14%),兆豐金 ( 0...",電腦及週邊設備業,6.34,"廣達 ( 1.65%),華碩 ( 0.82%),緯創 ( 0.62%),光寶科 ( 0.44..."
6,006203,半導體業,61.36,"台積電 ( 52.09%),聯發科 ( 4.05%),聯電 ( 1.08%),日月光投控 (...",金融保險,10.61,"富邦金 ( 1.53%),國泰金 ( 1.34%),中信金 ( 1.16%),兆豐金 ( 0...",電腦及週邊設備業,6.54,"廣達 ( 1.7%),華碩 ( 0.86%),緯創 ( 0.65%),光寶科 ( 0.44%..."
7,006204,半導體業,39.89,"台積電 ( 35.16%),聯發科 ( 3.55%),日月光投控 ( 1.18%)",金融保險,5.05,"富邦金 ( 2.1%),國泰金 ( 1.74%),中信金 ( 1.21%)",其他電子業,4.54,鴻海 ( 4.54%)
8,006208,半導體業,64.98,"台積電 ( 56.6%),聯發科 ( 4.35%),聯電 ( 1.29%),日月光投控 ( ...",金融保險,11.32,"富邦金 ( 1.68%),國泰金 ( 1.44%),中信金 ( 1.34%),兆豐金 ( 1...",其他電子業,5.75,鴻海 ( 5.75%)
9,00690,半導體業,59.41,"台積電 ( 33.25%),聯發科 ( 14.26%),聯電 ( 4.12%),聯詠 ( 2...",電子零組件業,12.88,"台達電 ( 6.25%),國巨 ( 1.89%),欣興 ( 1.52%),嘉澤 ( 1.15...",金融保險,7.90,"富邦金 ( 6.57%),上海商銀 ( 1.33%)"


In [11]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '320px'})
dfi.export(df_styled, "Result/mytable.png")

CalledProcessError: Command '['--enable-logging', '--disable-gpu', '--headless', '--crash-dumps-dir=C:\\Users\\user\\AppData\\Local\\Temp\\tmpf5rc84w2', '--force-device-scale-factor=1', '--window-size=1400,900', '--hide-scrollbars', '--screenshot=C:\\Users\\user\\AppData\\Local\\Temp\\tmpf5rc84w2\\temp.png', 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpf5rc84w2\\temp.html']' returned non-zero exit status 21.

In [12]:
df_styled

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,65.020000,"台積電 ( 56.63%),聯發科 ( 4.36%),聯電 ( 1.29%),日月光投控 ( 1.22%),聯詠 ( 0.65%),瑞昱 ( 0.52%),世芯-KY ( 0.35%)",金融保險,11.320000,"富邦金 ( 1.68%),國泰金 ( 1.44%),中信金 ( 1.34%),兆豐金 ( 1.05%),玉山金 ( 0.95%),元大金 ( 0.92%),第一金 ( 0.68%),合庫金 ( 0.63%),永豐金 ( 0.62%),凱基金 ( 0.6%),華南金 ( 0.59%),台新金 ( 0.48%),上海商銀 ( 0.34%)",其他電子業,5.680000,鴻海 ( 5.68%)
1,0051,電子零組件業,13.250000,"健策 ( 2.04%),台光電 ( 1.71%),嘉澤 ( 1.66%),群光 ( 1.29%),健鼎 ( 1.25%),川湖 ( 1.07%),臻鼎-KY ( 0.95%),華通 ( 0.92%),金像電 ( 0.78%),信邦 ( 0.76%),華新科 ( 0.44%),南電 ( 0.38%)",半導體業,12.760000,"矽力*-KY ( 2.25%),京元電子 ( 1.88%),創意 ( 1.48%),力成 ( 1.04%),祥碩 ( 0.91%),華邦電 ( 0.8%),力積電 ( 0.76%),智原 ( 0.71%),南亞科 ( 0.66%),旺宏 ( 0.58%),愛普* ( 0.54%),采鈺 ( 0.38%),達發 ( 0.35%),威盛 ( 0.32%),台勝科 ( 0.1%)",電腦及週邊設備業,9.740000,"仁寶 ( 1.96%),技嘉 ( 1.73%),微星 ( 1.66%),英業達 ( 1.58%),宏碁 ( 1.56%),佳世達 ( 0.66%),神基 ( 0.59%)"
2,0052,半導體業,77.880000,"台積電 ( 65.62%),聯發科 ( 5.07%),聯電 ( 1.5%),日月光投控 ( 1.4%),聯詠 ( 0.76%),瑞昱 ( 0.6%),矽力*-KY ( 0.44%),世芯-KY ( 0.41%),京元電子 ( 0.37%),創意 ( 0.28%),力成 ( 0.23%),祥碩 ( 0.18%),華邦電 ( 0.16%),力積電 ( 0.15%),智原 ( 0.14%),南亞科 ( 0.13%),旺宏 ( 0.11%),愛普* ( 0.1%),采鈺 ( 0.08%),達發 ( 0.07%),威盛 ( 0.06%),台勝科 ( 0.02%)",電腦及週邊設備業,8.320000,"廣達 ( 2.14%),華碩 ( 1.07%),緯創 ( 0.8%),和碩 ( 0.54%),光寶科 ( 0.54%),奇鋐 ( 0.48%),緯穎 ( 0.44%),研華 ( 0.39%),仁寶 ( 0.39%),技嘉 ( 0.34%),微星 ( 0.32%),英業達 ( 0.31%),宏碁 ( 0.31%),佳世達 ( 0.13%),神基 ( 0.12%)",其他電子業,7.510000,"鴻海 ( 6.69%),可成 ( 0.38%),鴻準 ( 0.24%),貿聯-KY ( 0.2%)"
3,0053,半導體業,64.340000,"台積電 ( 52.02%),聯發科 ( 4.56%),日月光投控 ( 1.35%),聯電 ( 1.25%),聯詠 ( 0.58%),創意 ( 0.44%),瑞昱 ( 0.43%),矽力*-KY ( 0.33%),京元電子 ( 0.25%),南亞科 ( 0.24%),聯鈞 ( 0.19%),力成 ( 0.17%),祥碩 ( 0.16%),華邦電 ( 0.16%),威盛 ( 0.12%),景碩 ( 0.1%),智原 ( 0.1%),台勝科 ( 0.09%),矽格 ( 0.09%),愛普* ( 0.09%),力積電 ( 0.08%),創見 ( 0.08%),義隆 ( 0.08%),華泰 ( 0.08%),旺宏 ( 0.08%),京鼎 ( 0.07%),聯陽 ( 0.07%),新唐 ( 0.07%),致新 ( 0.07%),同欣電 ( 0.06%),天鈺 ( 0.06%),瑞鼎 ( 0.06%),凌陽 ( 0.05%),超豐 ( 0.05%),強茂 ( 0.05%),矽創 ( 0.05%),力智 ( 0.04%),華東 ( 0.04%),南茂 ( 0.04%),盛群 ( 0.04%),立積 ( 0.04%),晶豪科 ( 0.04%),嘉晶 ( 0.04%),台亞 ( 0.04%),順德 ( 0.04%),偉詮電 ( 0.03%),菱生 ( 0.03%),敦泰 ( 0.03%),威鋒電子 ( 0.03%),昇陽半導體 ( 0.02%),松翰 ( 0.02%),矽統 ( 0.01%),光罩 ( 0.01%),界霖 ( 0.01%),福懋科 ( 0.01%)",電腦及週邊設備業,8.980000,"廣達 ( 2.26%),華碩 ( 0.81%),緯穎 ( 0.72%),緯創 ( 0.63%),光寶科 ( 0.51%),研華 ( 0.5%),和碩 ( 0.49%),奇鋐 ( 0.47%),技嘉 ( 0.36%),仁寶 ( 0.31%),英業達 ( 0.3%),微星 ( 0.29%),宏碁 ( 0.24%),佳世達 ( 0.15%),神基 ( 0.15%),神達 ( 0.11%),威強電 ( 0.07%),凌華 ( 0.07%),研揚 ( 0.07%),尼得科超眾 ( 0.06%),華擎 ( 0.06%),友通 ( 0.06%),飛捷 ( 0.05%),樺漢 ( 0.05%),勤誠 ( 0.05%),振樺電 ( 0.04%),輔信 ( 0.04%),精元 ( 0.03%),藍天 ( 0.01%),宏正 ( 0.01%),達方 ( 0.01%)",電子零組件業,8.040000,"台達電 ( 2.73%),國巨 ( 0.67%),欣興 ( 0.46%),嘉澤 ( 0.42%),川湖 ( 0.3%),健策 ( 0.29%),台光電 ( 0.27%),群光 ( 0.22%),臻鼎-KY ( 0.21%),健鼎 ( 0.2%),金像電 ( 0.2%),南電 ( 0.16%),信邦 ( 0.15%),華通 ( 0.14%),群電 ( 0.11%),新日興 ( 0.1%),華新科 ( 0.1%),毅嘉 ( 0.09%),建準 ( 0.09%),精成科 ( 0.08%),信錦 ( 0.08%),晶技 ( 0.08%),康舒 ( 0.07%),台郡 ( 0.06%),瀚宇博 ( 0.06%),日電貿 ( 0.06%),興勤 ( 0.06%),廣宇 ( 0.05%),敬鵬 ( 0.05%),全漢 ( 0.05%),聯茂 ( 0.05%),嘉聯益 ( 0.04%),健和興 ( 0.04%),志聖 ( 0.04%),立隆電 ( 0.04%),禾伸堂 ( 0.04%),嘉基 ( 0.03%),志超 ( 0.03%),凱美 ( 0.02%),鉅祥 ( 0.02%),大毅 ( 0.02%),飛宏 ( 0.01%),正崴 ( 0.01%),燿華 ( 0.01%),致伸 ( 0.01%),鎰勝 ( 0.01%),台虹 ( 0.01%)"
4,0056,電腦及週邊設備業,25.730000,"華碩 ( 3.56%),廣達 ( 2.95%),和碩 ( 2.86%),緯創 ( 2.66%),仁寶 ( 2.41%),光寶科 ( 2.35%),微星 ( 2.34%),英業達 ( 2.25%),技嘉 ( 1.87%),宏碁 ( 1.42%),神基 ( 1.06%)",半導體業,18.980000,"聯發科 ( 3.25%),聯詠 ( 3.17%),聯電 ( 3.01%),瑞昱 ( 2.93%),日月光投控 ( 2.62%),京元電子 ( 2.38%),力成 ( 1.37%),台勝科 ( 0.25%)",金融保險,13.160000,"兆豐金 ( 3.59%),中信金 ( 3.01%),永豐金 ( 2.91%),元大金 ( 2.86%),京城銀 ( 0.79%)"
5,0057,半導體業,60.550000,"台積電 ( 51.46%),聯發科 ( 3.94%),聯電 ( 1.1%),日月光投控 ( 1.05%),聯詠 ( 0.59%),瑞昱 ( 0.47%),力旺 ( 0.42%),矽力*-KY ( 0.33%),世芯-KY ( 0.32%),創意 ( 0.22%),環球晶 ( 0.22%),世界 ( 0.19%),華邦電 ( 0.13%),南亞科 ( 0.11%)",金融保險,10.560000,"富邦金 ( 1.51%),國泰金 ( 1.33%),中信金 ( 1.14%),兆豐金 ( 0.95%),玉山金 ( 0.8%),元大金 ( 0.68%),第一金 ( 0.62%),凱基金 ( 0.54%),合庫金 ( 0.54%),永豐金 ( 0.5%),華南金 ( 0.46%),台新金 ( 0.41%),新光金 ( 0.33%),上海商銀 ( 0.31%),彰銀 ( 0.22%),臺企銀 ( 0.22%)",電腦及週邊設備業,6.340000,"廣達 ( 1.65%),華碩 ( 0.82%),緯創 